# infos

In [ ]:
# Version 1.0

# Preambule

- Split de mushroom_observation en 3 :
    - **infos_images** pour lien images/label
    - **especesKey** pour variables 'Key'
    - **especes_classification** pour les autres variables

In [1]:
import pandas as pd
import os

#from google.colab import drive
#drive.mount('/content/drive')
#lien_observations_mushroom = '/content/drive/MyDrive/colab_champignons/csv/observations_mushroom.csv'
#mushroom_observation = pd.read_csv(lien_observations_mushroom, low_memory=False)
lien_csv = '../data/observations_mushroom.csv'
mushroom_observation = pd.read_csv(lien_csv, low_memory=False)


In [2]:
mushroom_observation.drop_duplicates(inplace=True)
mushroom_observation.duplicated().sum()

0

In [3]:
# Suppression des label représentés à moins de 0.1%
print('nb lignes initiales :', mushroom_observation.shape[0])
print('nb label initial :', mushroom_observation['label'].nunique())

compte_valeurs = mushroom_observation['label'].value_counts(normalize=True) * 100
seuil = 0.1
masque = mushroom_observation['label'].isin(compte_valeurs[compte_valeurs >= seuil].index)
mushroom_observation = mushroom_observation[masque]

print('\n nb lignes finales :', mushroom_observation.shape[0])
print('nb label final :', mushroom_observation['label'].nunique())

nb lignes initiales : 647615
nb label initial : 11999

 nb lignes finales : 207563
nb label final : 130


In [4]:
mushroom_observation.head()

,image_lien,image_id,observation,label,image_url,user,date,gbif_info/kingdom,gbif_info/family,gbif_info/speciesKey,...,gbif_info/phylumKey,gbif_info/class,gbif_info/synonym,gbif_info/scientificName,gbif_info/genus,gbif_info/order,thumbnail,location,gbif_info/note,gbif_info
12,15.jpg,15,12,Xerocomellus chrysenteron,http://mushroomobserver.org/images/320/15,1,2006-05-21 07:17:29,Fungi,Boletaceae,2519413.0,...,34.0,Agaricomycetes,False,"Xerocomellus chrysenteron (Bull.) Šutara, 2008",Xerocomellus,Boletales,1,4186.0,NaN,NaN
16,19.jpg,19,16,Fuligo septica,http://mushroomobserver.org/images/320/19,1,2006-05-21 07:17:34,Protozoa,Physaraceae,5426027.0,...,33.0,Myxomycetes,False,"Fuligo septica (L.) F.H.Wigg., 1780",Fuligo,Physarales,1,5.0,NaN,NaN
18,21.jpg,21,18,Agrocybe,http://mushroomobserver.org/images/320/21,1,2006-05-21 07:17:37,Fungi,Strophariaceae,NaN,...,34.0,Agaricomycetes,False,Agrocybe Fayod,Agrocybe,Agaricales,1,58.0,NaN,NaN
19,22.jpg,22,18,Agrocybe,http://mushroomobserver.org/images/320/22,1,2006-05-21 07:17:37,Fungi,Strophariaceae,NaN,...,34.0,Agaricomycetes,False,Agrocybe Fayod,Agrocybe,Agaricales,0,58.0,NaN,NaN
20,23.jpg,23,18,Agrocybe,http://mushroomobserver.org/images/320/23,1,2006-05-21 07:17:37,Fungi,Strophariaceae,NaN,...,34.0,Agaricomycetes,False,Agrocybe Fayod,Agrocybe,Agaricales,0,58.0,NaN,NaN


# infos_images

In [34]:
# Construction du dataset
infos_images = mushroom_observation[['label', 'image_lien']]

In [35]:
# Controle NaN
infos_images.isna().sum()

label         0
image_lien    0
dtype: int64

In [36]:
# Tri des labels
infos_images = infos_images.sort_values(by='label', ascending=True)
infos_images = infos_images.reset_index(drop=True)

In [37]:
# Ajout des URL des images

# Fonction pour créer le chemin complet des images
def ajout_url(image_lien):
    filepath = "../../images/"
    return os.path.join(filepath, image_lien)

# Utilisation de la méthode 'apply' pour créer la colonne 'image_url'
infos_images["image_url"] = infos_images["image_lien"].apply(ajout_url)

In [38]:
infos_images.drop_duplicates(inplace=True)
infos_images.shape[0]

207512

In [16]:
infos_images.head()

,label,image_lien,image_url
0,Agaricaceae,332044.jpg,../../images/332044.jpg
1,Agaricaceae,171135.jpg,../../images/171135.jpg
2,Agaricaceae,243640.jpg,../../images/243640.jpg
3,Agaricaceae,341497.jpg,../../images/341497.jpg
4,Agaricaceae,341496.jpg,../../images/341496.jpg


In [17]:
# Enregistrement du dataset
infos_images.to_csv('../data/infos_images.csv', index=False)

# TOP 10

In [39]:
# Le dataframe est basé sur infos_images qui lie le label au chemin vers l'image.
top10 = pd.read_csv('../data/infos_images.csv')

# On ne garde que les 10 espèces les plus représentées
nb_val = top10['label'].value_counts()
nb_val_label = nb_val.head(10)
nb_val_label = nb_val_label.index

top10 = top10[top10['label'].isin(nb_val_label)]
top10.reset_index(inplace=True, drop=True)

# Une des lignes renvoie vers une image inexistante qui poserait problème pour l'entrainement du modèle :
top10.drop(top10[top10['image_url'] == '../../images/611356.jpg'].index, axis=0, inplace=True)

top10.shape

(64372, 3)

In [40]:
# On retire toutes les lignes comportant des images qui ont été retirées du dataset (images poubelles)
tri_img = pd.read_csv('../../img_supp.csv', index_col = 0)
tri_img['fichiers'] = '../../images/' + tri_img['fichiers']
tri_img.head()
liste_supp = tri_img['fichiers'].tolist()

In [41]:
indices_a_supprimer = top10['image_url'].isin(liste_supp)
top10 = top10[~indices_a_supprimer]
top10.shape

(60481, 3)

In [44]:
top10.to_csv('../data/top10.csv', index=False)

# especes_key

In [45]:
# Construction du dataset
especes_key = mushroom_observation[['label', 'date', 'gbif_info/kingdomKey', 'gbif_info/familyKey', 'gbif_info/speciesKey',
                                   'gbif_info/genusKey', 'gbif_info/phylumKey', 'gbif_info/orderKey',
                                   'gbif_info/classKey', 'gbif_info/usageKey']]

In [46]:
# Suppression du prefixe gbif_info/ sur les colonnes
supp_prefixe_col = {
    'gbif_info/speciesKey': 'speciesKey',
    'gbif_info/orderKey': 'orderKey',
    'gbif_info/classKey': 'classKey',
    'gbif_info/familyKey' : 'familyKey',
    'gbif_info/usageKey' : 'scientificNameKey',
    'gbif_info/kingdomKey' : 'kingdomKey',
    'gbif_info/genusKey' : 'genusKey',
    'gbif_info/phylumKey': 'phylumKey'}

especes_key = especes_key.rename(columns = supp_prefixe_col)

In [47]:
# Colonne 'date' :  on ne garde que le mois où l'observation a été réalisée. 
# On discrétise ensuite les mois dans une variable 'saison'.

# Suppression des hh:mm:ss de la colonne 'date' :
especes_key['date'] = especes_key['date'].str.slice(start=5, stop=7)

# Construction d'une fonction retournant la saison en fonction du mois observé
def get_season(date):
    if date == '12' or date == '01' or date == '02':
        return '1' # Hiver
    elif date == '03' or date == '04' or date == '05':
        return '2' # Printemps
    elif date == '06' or date == '07' or date == '08':
        return '3' # Ete
    else:
        return '4' # Automne

# Construction de la colonne 'saison' en appliquant la fonction 'get_season' sur la colonne 'date'
especes_key['saison'] = especes_key['date'].apply(get_season)

# Application du type 'int' à la colonne 'saison' :
especes_key['saison'] = especes_key['saison'].astype('int64')

# Suppression de la colonne 'date' :
especes_key.drop('date', axis = 1, inplace = True)

In [48]:
# Controle NaN
especes_key.isna().sum()

label                     0
kingdomKey            20914
familyKey             43137
speciesKey           158667
genusKey              49498
phylumKey             23957
orderKey              24871
classKey              23957
scientificNameKey     20914
saison                    0
dtype: int64

In [49]:
# Tri des labels
especes_key.sort_values(by='label', ascending=True, inplace=True)
especes_key = especes_key.reset_index(drop=True)

In [50]:
# Chaque nom d'espèce va porter un id. Il reste 130 espèces enregistrées, soit des id allant de 0 à 129 :
especes_key['label_id'] = pd.factorize(especes_key['label'])[0]


In [51]:
especes_key.head()

,label,kingdomKey,familyKey,speciesKey,genusKey,phylumKey,orderKey,classKey,scientificNameKey,saison,label_id
0,Agaricaceae,5.0,4170.0,NaN,NaN,34.0,1499.0,186.0,4170.0,2,0
1,Agaricaceae,5.0,4170.0,NaN,NaN,34.0,1499.0,186.0,4170.0,4,0
2,Agaricaceae,5.0,4170.0,NaN,NaN,34.0,1499.0,186.0,4170.0,3,0
3,Agaricaceae,5.0,4170.0,NaN,NaN,34.0,1499.0,186.0,4170.0,3,0
4,Agaricaceae,5.0,4170.0,NaN,NaN,34.0,1499.0,186.0,4170.0,3,0


In [52]:
especes_key.shape[0]

207563

In [53]:
# Enregistrement du dataset
especes_key.to_csv('../data/especes_key.csv', index=False)

# especes_classification

In [54]:
# Construction du dataset
especes_classification = mushroom_observation[['label', 'date', 'gbif_info/kingdom', 'gbif_info/family',
                                                'gbif_info/phylum', 'gbif_info/species', 'gbif_info/class',
                                                'gbif_info/scientificName', 'gbif_info/genus', 'gbif_info/order']]

In [55]:
# Suppression du prefixe gbif_info/ sur les colonnes
supp_prefixe_col = {
    'gbif_info/kingdom': 'kingdom',
    'gbif_info/family': 'family',
    'gbif_info/phylum': 'phylum',
    'gbif_info/species': 'species',
    'gbif_info/class': 'class',
    'gbif_info/scientificName': 'scientificName',
    'gbif_info/genus': 'genus',
    'gbif_info/order': 'order'}

especes_classification = especes_classification.rename(columns = supp_prefixe_col)

In [56]:
# Colonne 'date' :  on ne garde que le mois où l'observation a été réalisée. 
# On discrétise ensuite les mois dans une variable 'saison'.

# Suppression des hh:mm:ss de la colonne 'date' :
especes_classification['date'] = especes_classification['date'].str.slice(start=5, stop=7)

# Construction d'une fonction retournant la saison en fonction du mois observé
def get_season(date):
    if date == '12' or date == '01' or date == '02':
        return '1' # Hiver
    elif date == '03' or date == '04' or date == '05':
        return '2' # Printemps
    elif date == '06' or date == '07' or date == '08':
        return '3' # Ete
    else:
        return '4' # Automne

# Construction de la colonne 'saison' en appliquant la fonction 'get_season' sur la colonne 'date'
especes_classification['saison'] = especes_classification['date'].apply(get_season)

# Application du type 'int' à la colonne 'saison' :
especes_classification['saison'] = especes_classification['saison'].astype('int64')

# Suppression de la colonne 'date' :
especes_classification.drop('date', axis = 1, inplace = True)

In [57]:
# Controle NaN
especes_classification.isna().sum()

label                  0
kingdom            20914
family             43137
phylum             23957
species           158667
class              23957
scientificName     20914
genus              49498
order              24871
saison                 0
dtype: int64

In [58]:
# Tri des labels
especes_classification.sort_values(by='label', ascending=True, inplace=True)
especes_classification = especes_classification.reset_index(drop=True)

In [59]:
especes_classification.head()

,label,kingdom,family,phylum,species,class,scientificName,genus,order,saison
0,Agaricaceae,Fungi,Agaricaceae,Basidiomycota,NaN,Agaricomycetes,Agaricaceae,NaN,Agaricales,2
1,Agaricaceae,Fungi,Agaricaceae,Basidiomycota,NaN,Agaricomycetes,Agaricaceae,NaN,Agaricales,4
2,Agaricaceae,Fungi,Agaricaceae,Basidiomycota,NaN,Agaricomycetes,Agaricaceae,NaN,Agaricales,3
3,Agaricaceae,Fungi,Agaricaceae,Basidiomycota,NaN,Agaricomycetes,Agaricaceae,NaN,Agaricales,3
4,Agaricaceae,Fungi,Agaricaceae,Basidiomycota,NaN,Agaricomycetes,Agaricaceae,NaN,Agaricales,3


In [60]:
especes_classification.shape[0]

207563

In [61]:
# Enregistrement du dataset
especes_classification.to_csv('../data/especes_classification.csv', index=False)